In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorboard
from keras.callbacks import Callback, TensorBoard
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_image='D:/Documents/Capstone/Project -2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [4]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 47454 images belonging to 35 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=20)

Found 14321 images belonging to 35 classes.


In [6]:
#Mobilenet
mobileNet = "https://tfhub.dev/google/imagenet/mobilenet_v2_050_224/classification/5"

In [7]:
# model 
model = Sequential([
    #layer input
    hub.KerasLayer(mobileNet, input_shape=(224, 224, 3), trainable=False),
    #layer output
    Dense(35, activation="softmax")]
)

In [8]:
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              1988505   
                                                                 
 dense (Dense)               (None, 35)                35070     
                                                                 
Total params: 2,023,575
Trainable params: 35,070
Non-trainable params: 1,988,505
_________________________________________________________________


In [9]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [10]:
batch_size=512
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15,
                    callbacks=[early])

Epoch 1/15
9490/9490 [==============================] - 4723s 494ms/step - loss: 0.4732 - accuracy: 0.8717 - auc: 0.9875 - precision: 0.8942 - recall: 0.8579 - true_positives: 40706.0000 - val_loss: 8.4061 - val_accuracy: 0.6129 - val_auc: 0.8362 - val_precision: 0.6185 - val_recall: 0.6095 - val_true_positives: 8728.0000
Epoch 2/15
9490/9490 [==============================] - 5381s 567ms/step - loss: 0.3609 - accuracy: 0.9145 - auc: 0.9886 - precision: 0.9197 - recall: 0.9115 - true_positives: 43248.0000 - val_loss: 11.0532 - val_accuracy: 0.6339 - val_auc: 0.8388 - val_precision: 0.6380 - val_recall: 0.6321 - val_true_positives: 9052.0000
Epoch 3/15
9490/9490 [==============================] - 5456s 575ms/step - loss: 0.3412 - accuracy: 0.9260 - auc: 0.9884 - precision: 0.9294 - recall: 0.9242 - true_positives: 43854.0000 - val_loss: 12.7334 - val_accuracy: 0.6348 - val_auc: 0.8369 - val_precision: 0.6383 - val_recall: 0.6332 - val_true_positives: 9067.0000
Epoch 4/15
9490/9490 [====


KeyboardInterrupt



In [11]:
from matplotlib import pyplot
pyplot.plot(history.history['accuracy'])
pyplot.show()

NameError: name 'history' is not defined